In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import edurel.utils.question_man as qm
import edurel.utils.conversation as conv
import edurel.utils.duckdb as ddb
import edurel.utils.sql_agent as sa


load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
QUESTIONS_DIR = f"{BASE_DIR}/questions"
CONFIG_DIR = f"{BASE_DIR}/config"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [3]:
db = ddb.Db.mem("company_en")
db.config_exe(f"{CONFIG_DIR}/databases/company_en.yaml")

# Questions

In [4]:
qman = qm.QuestionMan(f"{QUESTIONS_DIR}/company_en.txt")

In [17]:
sqlagent = sa.SQLAgent(
    db,
    conv.Client.STATS, 
    conv.LLMName.GLM47,
    fk_spec_file_path=f"{CONFIG_DIR}/fk_specs/company_en.yaml",
    rem_tags_spec_file_path=f"{CONFIG_DIR}/omit_tags_specs/dbstuff.yaml",
    log_dir=f"{BASE_DIR}/logs",
    autolog=False
)

# Text2SQL

In [19]:
for q in qman[2:10]:
    print(q[0])
    sqlagent.text2sql(q[0], q[1])
    # sqlagent.conversation.display()
    sqlagent.log(q[0], "v1")
    sqlagent.conversation.delete_slice(2)


hireyears_dept_14
employees_without_bonus
employees_with_bonus
total_income_per_employe_dept_17
salary_band_per_employee
employees_with_orgunit_name
employees_per_project
parent_child_org


In [8]:
sqlagent.text2sql(qman[0][0], qman[0][1])

In [12]:
sqlagent.text2sql(qman[1][0], qman[1][1])


In [15]:
sqlagent.conversation.delete_slice(2)


3

In [16]:
sqlagent.conversation.display(lastn_only=4)

[0] SYSTEM:

You are an expert SQL query generator. Convert natural language questions into valid SQL queries. Use duckdb syntax.

[1] USER:

Database Schema:
```yaml
tables:
- tablename: EmpProj
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: PID
    type: INTEGER
  - columnname: NoOfHoursPerWeek
    type: INTEGER
  primary_key:
  - EID
  - PID
  foreign_keys:
  - sourcecolumns:
    - EID
    targettable: Employee
    targetcolumns:
    - EID
  - sourcecolumns:
    - PID
    targettable: Project
    targetcolumns:
    - PID
- tablename: Employee
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: OUID
    type: INTEGER
  - columnname: LastName
    type: VARCHAR
  - columnname: Hiredate
    type: DATE
  - columnname: Salary
    type: DECIMAL(9,2)
  - columnname: Bonus
    type: DECIMAL(9,2)
  primary_key:
  - EID
  foreign_keys:
  - sourcecolumns:
    - OUID
    targettable: OrgUnit
    targetcolumns:
    - OUID
- tablename: OrgUnit
  columns:
  - columnname: OUID
    type: INTEGER
  - columnname: Head
    type: INTEGER
  - columnname: SuperUnit
    type: INTEGER
  - columnname: Name
    type: VARCHAR
  primary_key:
  - OUID
  foreign_keys:
  - sourcecolumns:
    - Head
    targettable: Employee
    targetcolumns:
    - EID
  - sourcecolumns:
    - SuperUnit
    targettable: OrgUnit
    targetcolumns:
    - OEID
- tablename: Project
  columns:
  - columnname: PID
    type: INTEGER
  - columnname: Title
    type: VARCHAR
  - columnname: Budget
    type: DECIMAL(13,2)
  primary_key:
  - PID

```

In [7]:
sqlagent.log(qman[0][0], "v1")

In [14]:
print(qman[0][1])

create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 

sorted by eid


In [12]:
for q in qman[:]:
    print(f"---tag: {q[0]}")
    print(q[1])

---tag: entry_year
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 

sorted by eid
---tag: entry_year_dept_14
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 

only for department 14

sorted by eid
---tag: hireyears_dept_14
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- number of years of deployment per employee 

only for department 14

sorted by eid
---tag: employees_without_bonus
create a SQL query that outputs the following columns:
- lastname, salary, bonus from table employee

only employees without bonus

sorted by eid
---tag: employees_with_bonus
create a SQL query that outputs the following columns:
- lastname, salary, bonus from table employee

only employees with bonus

sorted by eid
---tag: total_income_per_employe_dept_17
create a SQL query that outputs the follo